[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/data-platforms/aryn/weaviate_blog_post.ipynb)

## Chonkie and WeaviateHandshake Example

This notebook demonstrates how to use Chonkie's WeaviateHandshake to process markdown documents, extract chunks (text and tables), and ingest them into Weaviate to search and natural language querying.

## Install the Dependencies 

In [5]:
!pip install "chonkie[all]" --q

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=15912) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install weaviate-client --q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
!pip install weaviate-agents --q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Import the Libraries and API Keys

In [10]:
import weaviate
from weaviate.classes.config import Configure, Property, DataType
from weaviate.agents.query import QueryAgent

import chonkie
from chonkie import MarkdownChef
from chonkie import WeaviateHandshake
from chonkie import Chunk   


from pathlib import Path
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""

## Ingest Chunks into Weaviate

#### Connect to Weaviate Cloud 

You can create a free 14-day sandbox on [Weaviate Cloud](https://console.weaviate.cloud)!

In [15]:
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
    headers={
    "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),
    }
)

#### Create the Collection

You can skip this step if you:
1. Have an existing Weaviate collection using `text2vec_openai`
2. Want to use an open-source model instead of OpenAI

In [ ]:
# Create the collection

weaviate_client.collections.create(
    "WeaviateBlogs",
    vector_config=Configure.Vectors.text2vec_openai(
        model="text-embedding-3-small"
    ),
    properties=[
        Property(
            name="text",
            data_type=DataType.TEXT
        ),
        Property(
            name="chunk_type",
            data_type=DataType.TEXT,
            index_searchable=False,
            index_filterable=True
        )
    ]
)

#### Initialize the Weaviate Handshake

In [24]:
handshake = WeaviateHandshake(
    url=os.getenv("WEAVIATE_URL"),
    api_key=os.getenv("WEAVIATE_API_KEY"),
    collection_name="WeaviateBlogs",
    embedding_model="openai://text-embedding-3-small"
)

### Create the Chunks

In [ ]:
# Set up path to the directory of markdown files
path = Path("../../llm-agent-frameworks/data")

processed_documents = []
chunks: list[Chunk] = []

# Process all markdown files in the directory and subdirectories
for md_file in path.rglob("*.mdx"):
    print(f"Processing {md_file}...")
    chef = MarkdownChef()
    processed_document = chef.process(str(md_file))  # convert Path to str!
    processed_documents.append(processed_document)
    chunks.extend(processed_document.chunks)
    for table in processed_document.tables:
        chunks.extend([
            Chunk(text=table.content)
        ])
    for code_block in processed_document.code:
        chunks.extend([
            Chunk(text=code_block.content)
        ])

2025-11-12 12:13:55.849 | DEBUG    | chonkie.chef.base:read:68 - Reading file: /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-02-07-generative-search/index.mdx
2025-11-12 12:13:55.850 | DEBUG    | chonkie.chef.base:read:71 - Successfully read file: /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-02-07-generative-search/index.mdx
2025-11-12 12:13:55.850 | DEBUG    | chonkie.chef.markdown:parse:200 - Processing markdown text: 13934 characters
2025-11-12 12:13:55.851 | INFO     | chonkie.chef.markdown:parse:210 - Markdown processing complete: extracted 1 tables, 6 code blocks, 2 images, 10 chunks
2025-11-12 12:13:55.852 | DEBUG    | chonkie.chef.base:read:68 - Reading file: /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-02-21-combining-langchain-and-weaviate/index.mdx
2025-11-12 12:13:55.852 | DEBUG    | chonkie.chef.base:read:71 - Successfully read file: /Users/erikac

Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-02-07-generative-search/index.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-02-21-combining-langchain-and-weaviate/index.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-01-10-pulling-back-the-curtains-on-text2vec/index.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2025-04-17-graph-rag/index.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-05-04-weaviate-1-19-release/index.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-05-04-weaviate-1-19-release/_core-1-19-include.mdx...
Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-12-19-weaviate-1-23-release/index.mdx

2025-11-12 12:13:56.049 | INFO     | chonkie.chef.markdown:parse:210 - Markdown processing complete: extracted 0 tables, 12 code blocks, 2 images, 14 chunks
2025-11-12 12:13:56.050 | DEBUG    | chonkie.chef.base:read:68 - Reading file: /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-03-23-what-are-llms/index.mdx
2025-11-12 12:13:56.050 | DEBUG    | chonkie.chef.base:read:71 - Successfully read file: /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-03-23-what-are-llms/index.mdx
2025-11-12 12:13:56.051 | DEBUG    | chonkie.chef.markdown:parse:200 - Processing markdown text: 11168 characters
2025-11-12 12:13:56.051 | INFO     | chonkie.chef.markdown:parse:210 - Markdown processing complete: extracted 0 tables, 0 code blocks, 6 images, 7 chunks


Processing /Users/erikacardenas/Desktop/GitHub/recipes/integrations/llm-agent-frameworks/data/2023-03-23-what-are-llms/index.mdx...


In [76]:
print(len(chunks))

974


In [77]:
print(f"\nProcessed {len(processed_documents)} documents")
print(f"Total chunks: {len(chunks)}")
num_tables = sum(len(doc.tables) for doc in processed_documents)
print(f"Found {num_tables} tables")

num_code_blocks = sum(len(doc.code) for doc in processed_documents)
print(f"Found {num_code_blocks} code blocks")

num_images = sum(len(doc.images) for doc in processed_documents)
print(f"Found {num_images} images")

num_text_chunks = sum(len(doc.chunks) for doc in processed_documents)
print(f"Found {num_text_chunks} text chunks")


Processed 78 documents
Total chunks: 974
Found 37 tables
Found 258 code blocks
Found 356 images
Found 679 text chunks


### Write the Chunks to Weaviate Collection

In [78]:
handshake.write(chunks)

2025-11-12 12:14:01.751 | DEBUG    | chonkie.handshakes.weaviate:write:320 - Writing 974 chunks to Weaviate collection: WeaviateBlogs
2025-11-12 12:20:13.111 | INFO     | chonkie.handshakes.weaviate:write:375 - Successfully wrote 974 chunks to Weaviate collection: WeaviateBlogs


🦛 Chonkie wrote 974 chunks to Weaviate collection: WeaviateBlogs


['2397c169-832b-5d30-9039-3ab3e6d8c9fa',
 'cce18011-0e3f-57ba-ac61-1b3ad8c7d3b4',
 'f4ffad39-b32f-5db3-8060-f59581e9f575',
 '3238b25d-0d39-5e22-94f9-764aa060cb4d',
 '1fd0612b-1dd5-5407-9c3e-fefb031b9547',
 '6cb29480-8a13-577f-934a-59bcc6b20875',
 '81dc395e-5784-5204-a2ca-830ebccfae2f',
 'e512cf00-8ed1-58d8-9e79-21bf0cec6111',
 'daff102b-1645-5109-937f-b8b30de23f4d',
 'f70b1e26-a5ce-53a5-808c-b34066669d5e',
 '11bc115a-ed7a-5e70-ac4c-1d5f2f3fe380',
 'cca106ef-2ecd-5e37-bae4-97b2fc0473f4',
 '161a8d2e-d0ff-522c-98ef-64c68108ff40',
 '00ff25e3-b27f-5c5b-8296-22dc96ae7203',
 'e050a26e-ce8c-51b1-a706-86fffba69598',
 'ef078f7d-b078-5702-9360-3b63dd9810dd',
 'be993a27-fa3e-57a4-95d6-4dfda78051db',
 'dff164b3-f263-563d-8f12-8de7a82d2dd3',
 '256e60ba-3c75-566d-9618-16ac84da2b4a',
 '2ba30093-f2d4-50ab-8d14-1e07b32b6082',
 '4985168d-a5b6-56e6-9c48-055b68943217',
 'bef6e5a2-48b8-5e3f-badd-1cfd2357a5d9',
 '08a02544-926a-59ac-8d86-8389e02ecbaa',
 '4455cc70-9159-5af6-bd63-8119d54f5d69',
 '7ccff1c6-8ace-

## Query Time

### Search the Chunks using Chonkie

In [25]:
results = handshake.search(query="explain the concept of agentic rag", limit=2)
for result in results:
    print(result["score"], result["text"])

2025-11-13 13:11:50.608 | DEBUG    | chonkie.handshakes.weaviate:search:467 - Searching Weaviate collection: WeaviateBlogs with limit=2
2025-11-13 13:11:51.975 | INFO     | chonkie.handshakes.weaviate:search:495 - Search complete: found 2 matching chunks


0.7075777053833008 ---
title: What is Agentic RAG
slug: what-is-agentic-rag
authors: [erika, leonie]
date: 2024-11-05
tags: ['concepts', 'agents']
image: ./img/hero.png
description: 'Learn about Agentic Retrieval Augmented Generation (RAG), including architecture, implementation, and and difference to vanilla RAG.'
---

0.6376795768737793 
    

## What is Agentic RAG?

Agentic RAG describes an AI agent-based implementation of RAG. Specifically, it incorporates AI agents into the RAG pipeline to orchestrate its components and perform additional actions beyond simple information retrieval and generation to overcome the limitations of the non-agentic pipeline.

> Agentic RAG describes an AI agent-based implementation of RAG.

### How does Agentic RAG work?

Although agents can be incorporated in different stages of the RAG pipeline, agentic RAG most commonly refers to the use of agents in the retrieval component. 

Specifically, the retrieval component becomes agentic through the use of 

### Weaviate Query Agent

The [Weaviate Query Agent](https://docs.weaviate.io/agents/query/usage) can be used to answer natural language queries based on the objects in our database. 

#### Search Mode

[`Search` mode](https://docs.weaviate.io/agents/query/usage#search) will process the question, run the searches, and return the relevant objects.

In [29]:
qa = QueryAgent(
    client=weaviate_client,
    collections=["WeaviateBlogs"],
)

response = qa.search(
    "Data comparing Weaviate Quantization methods.",
    limit=1
)

for o in response.search_results.objects:
    props = o.properties
    print(props)

{'text': '\n\nWeaviate improves [binary quantization (BQ)](/developers/weaviate/configuration/compression/bq-compression) in 1.24 to be faster, more memory efficient, and more cost-effective. Use BQ vector compression with [HNSW indexes](/developers/weaviate/concepts/vector-index#hierarchical-navigable-small-world-hnsw-index) to dramatically improve your query speed.\n\nBQ compresses vector representations while preserving essential information. Uncompressed, Weaviate uses a `float32` to store each dimension. BQ uses one bit per dimension to encode Vector directionality. This means BQ compresses vectors from 32 bits per dimension to 1 bit per dimension - a savings of 32 times the space. This compression significantly reduces storage requirements.\n\nComparing BQ compressed vectors is fast. To calculate the distance between to vectors, compare their dimensional agreement. The bitwise comparison ensures optimal resource utilization without compromising accuracy. For instance, consider tw

#### Ask Mode

[`Ask` mode](https://docs.weaviate.io/agents/query/usage#ask) will process the question, run the searches, and return the generated answer.

In [20]:
qa = QueryAgent(
    client=weaviate_client,
    collections=["WeaviateBlogs"]
)

response = qa.ask("Data comparing Weaviate Quantization methods")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ Weaviate implements two main quantization methods for vector compression and efficient storage with tradeoffs   │
│ in speed, memory usage, and accuracy:                                                                           │
│                                                                                                                 │
│ 1. **Binary Quantization (BQ):**                                                                                │
│    - Introduced and improved in Weaviate v1.24.                                                                 │
│    - Compresses vectors by encoding each dimension using one bit representing directionality rather than 32-bit │
│ floats, achieving a 32x reduction in storage.                                                                   │
│    - Enables very fast, low-memory vector comparisons using bitwise operations.                                 │
│    - Often used with HNSW indexes for fast approximate nearest neighbor search.                                 │
│    - Provides significant query speed improvements and reduced storage while preserving essential vector        │
│ similarity information.                                                                                         │
│    - Often better suited for higher dimensional vectors.                                                        │
│    - BQ is also supported in the new flat index type, which is disk-based and suitable for smaller multi-tenant │
│ collections with minimal memory footprint.                                                                      │
│    - Weaviate may compensate for any loss in accuracy by over-fetching and re-ranking results.                  │
│                                                                                                                 │
│ 2. **Product Quantization (PQ):**                                                                               │
│    - Introduced in v1.18.                                                                                       │
│    - A lossy compression technique that splits vectors into segments and compresses these segments,             │
│ significantly reducing the vector size.                                                                         │
│    - Comes with a tradeoff of a slight decrease in recall accuracy.                                             │
│    - Integrated with HNSW indexing to keep memory usage low while maintaining good query performance.           │
│    - Weaviate 1.23 added AutoPQ, an automated training and segment sizing feature to simplify PQ setup and      │
│ optimize compression.                                                                                           │
│    - PQ compresses vectors in a way that requires new or adapted distance functions but strives to keep         │
│ distortion minimal to maintain retrievability.                                                                  │
│                                                                                                                 │
│ These methods offer different balances of compression ratio, speed, memory efficiency, and accuracy. BQ         │
│ maximizes speed and storage savings but may lose some vector detail, best for high-dimensional vectors and fast │
│ searches. PQ offers a more nuanced compression with some loss in recall but keeps memory needs down without     │
│ requiring decompression.                                                                                        │
│                                                                                                                 │
│ Weaviate lets users choose between these quantization t

╭───────────────────────────────────────────────── 🔭 Search 1/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='comparison of Weaviate quantization methods',                                                        │
│     filters=None,                                                                                               │
│     collection='WeaviateBlogs',                                                                                 │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 2/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='Weaviate quantization methods explained',                                                            │
│     filters=None,                                                                                               │
│     collection='WeaviateBlogs',                                                                                 │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 3/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='quantization techniques in Weaviate vector database',                                                │
│     filters=None,                                                                                               │
│     collection='WeaviateBlogs',                                                                                 │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='3307d39f-7648-5568-b837-7d615ac724ea' collection='WeaviateBlogs'                                  │
│  - object_id='a9c439aa-88ff-5a11-a5b8-c645d965fc39' collection='WeaviateBlogs'                                  │
│  - object_id='5eb7f2f5-f94d-592c-84e7-83e59fb14a33' collection='WeaviateBlogs'                                  │
│  - object_id='c03c7854-1aa3-50f1-8b31-68f55d3ddc5e' collection='WeaviateBlogs'                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 495  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ -1   │
└──────────────────────────┴──────┘

Total Time Taken: 12.27s